# Imports

In [1]:
import sys
!{sys.executable} -m pip install lyricsgenius

     |████████████████████████████████| 51kB 2.5MB/s 


In [2]:
# Libraries for extracting, cleaning, and manipulating data

from helpers import *
import pandas as pd
import numpy as np
import string

# For plotting graphs

from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Library used to count the frequency of words 

from sklearn.feature_extraction.text import CountVectorizer

# To create the sentiment analysis model, tokenization and lemmatization

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')

# To create the Topic Model

from helpers import lyrics_to_words
import gensim
from gensim import corpora
from gensim.corpora.dictionary import Dictionary


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Generate metric for sentiment based on all Trump tweets (generated from Bidirectional LSTM)

In [3]:
import pandas as pd
import numpy as np

tdf = pd.read_csv('trump.csv')
sent =  tdf['sentiment']

unique, counts = np.unique(sent, return_counts=True)
count = dict(zip(unique, counts))

total = count['0'] + count['1'] + count['2']

dif = count['2'] - count['0']
metric = dif/total
print(dif/total)

0.22553744517246982


In [4]:
# Extracting the information of the 100 most popular songs of Coldplay using function created on helpers script

access_token = 'vLPkp7ZMkcwo9wierjo2I_W_9bcUPxJcJZbZAjToQYvPeTInYQf8AEtMxtqiVY7t'
df0 = search_data('Coldplay',100,access_token)

Searching for songs by Coldplay...

Song 1: "The Scientist"
Song 2: "Viva la Vida"
Song 3: "Fix You"
Song 4: "Yellow"
Song 5: "Hymn for the Weekend"
Song 6: "A Sky Full of Stars"
Song 7: "Everglow"
Song 8: "Adventure of a Lifetime"
Song 9: "Orphans"
Song 10: "Paradise"
Song 11: "Magic"
Song 12: "Clocks"
Song 13: "Daddy"
Song 14: "Everyday Life"
Song 15: "Sparks"
Song 16: "Up&Up"
Song 17: "Trouble"
Song 18: "Arabesque"
Song 19: "Hypnotised"
Song 20: "All I Can Think About Is You"
Song 21: "Fly On"
Song 22: "Champion of the World"
Song 23: "Violet Hill"
Song 24: "Ink"
Song 25: "In My Place"
Song 26: "Church"
Song 27: "Always in My Head"
Song 28: "Midnight"
Song 29: "Shiver"
Song 30: "Strawberry Swing"
Song 31: "Trouble in Town"
Song 32: "True Love"
Song 33: "O"
Song 34: "A Rush of Blood to the Head"
Song 35: "Green Eyes"
Song 36: "Don’t Panic"
Song 37: "Every Teardrop Is a Waterfall"
Song 38: "Charlie Brown"
Song 39: "Fun"
Song 40: "Amazing Day"
Song 41: "Speed of Sound"
Song 42: "A Head

In [5]:
df0.head(10)

,artist,title,album,date,lyric
0,Coldplay,The Scientist,A Rush of Blood to the Head,2002-08-26,"[Verse 1]\nCome up to meet you, tell you I'm s..."
1,Coldplay,Viva la Vida,Viva La Vida or Death and All His Friends,2008-05-25,[Verse 1: Chris Martin]\nI used to rule the wo...
2,Coldplay,Fix You,X&Y,2005-06-06,[Verse 1: Chris Martin]\nWhen you try your bes...
3,Coldplay,Yellow,Parachutes,2000-06-26,[Verse 1: Chris Martin]\nLook at the stars\nLo...
4,Coldplay,Hymn for the Weekend,A Head Full of Dreams,2016-01-25,"[Intro: Beyoncé]\nAnd said drink from me, drin..."
5,Coldplay,A Sky Full of Stars,Ghost Stories,2014-05-02,"[Verse 1]\n'Cause you're a sky, 'cause you're ..."
6,Coldplay,Everglow,A Head Full of Dreams,2015-11-26,"[Verse 1]\nOh, they say people come, say peopl..."
7,Coldplay,Adventure of a Lifetime,A Head Full of Dreams,2015-11-06,[Intro]\n[Indecipherable]\n\n[Verse 1]\nTurn y...
8,Coldplay,Orphans,Everyday Life,2019-10-24,"[Intro: Chris Martin & Choir, Moses Martin]\nI..."
9,Coldplay,Paradise,Mylo Xyloto,2011-09-12,[Instrumental Intro]\n\n[Verse 1]\nOoh-ooh-ooh...


# Cleaning and transforming the data using functions created on helpers script

In [6]:
df = clean_lyrics(df0, 'lyric')
#df = create_decades(df)

# Filter data to only use songs that have lyrics
df = df[df['lyric'].notnull()]

# Exporting data into a csv file
df.to_csv('lyrics.csv', index=False)

In [7]:
df.head(10)

,artist,title,album,date,lyric
0,Coldplay,The Scientist,A Rush of Blood to the Head,2002-08-26,come up to meet you tell you i'm sorry you don...
1,Coldplay,Viva la Vida,Viva La Vida or Death and All His Friends,2008-05-25,chris martin i used to rule the world seas wou...
2,Coldplay,Fix You,X&Y,2005-06-06,chris martin when you try your best but you do...
3,Coldplay,Yellow,Parachutes,2000-06-26,chris martin look at the stars look how they s...
4,Coldplay,Hymn for the Weekend,A Head Full of Dreams,2016-01-25,beyoncé and said drink from me drink from me o...
5,Coldplay,A Sky Full of Stars,Ghost Stories,2014-05-02,'cause you're a sky 'cause you're a sky full o...
6,Coldplay,Everglow,A Head Full of Dreams,2015-11-26,oh they say people come say people go this par...
7,Coldplay,Adventure of a Lifetime,A Head Full of Dreams,2015-11-06,indecipherable turn your magic on umi she'd ...
8,Coldplay,Orphans,Everyday Life,2019-10-24,chris martin choir moses martin i want to kno...
9,Coldplay,Paradise,Mylo Xyloto,2011-09-12,oohoohooh oohoohooh oohoohooh oohoohooh oohooh...


# Sentiment Analysis using VADER Sentiment Intensity Model

In [8]:
# Creating lists to store the different scores for each word

negative = []
neutral = []
positive = []
compound = []
dist = []

# Initializing the model

sid = SentimentIntensityAnalyzer()

# Iterating for each row of lyrics and append the scores

for i in df.index:
    scores = sid.polarity_scores(df['lyric'].iloc[i])
    negative.append(scores['neg'])
    neutral.append(scores['neu'])
    positive.append(scores['pos'])
    compound.append(scores['compound'])
    dist.append(abs(scores['compound'] - metric))

# Creating 4 columns to the main data frame  for each score 

df['negative'] = negative
df['neutral'] = neutral
df['positive'] = positive
df['compound'] = compound
df['dist'] = dist

new_df = df.sort_values(by=['dist'])

new_df.head(40)

,artist,title,album,date,lyric,negative,neutral,positive,compound,dist
0,Coldplay,The Scientist,A Rush of Blood to the Head,2002-08-26,come up to meet you tell you i'm sorry you don...,0.101,0.809,0.091,0.2330,0.007463
89,Coldplay,WOTW / POTP,Everyday Life,2019-11-22,in a world gone wrong i shall be strong wonder...,0.097,0.806,0.097,0.2373,0.011763
26,Coldplay,Always in My Head,Ghost Stories,2014-05-16,duction i think of you i haven't slept i thin...,0.000,0.976,0.024,0.2498,0.024263
19,Coldplay,All I Can Think About Is You,Kaleidoscope [EP],2017-07-14,one two three fish fell out of water bird st...,0.091,0.816,0.092,0.2732,0.047663
40,Coldplay,Speed of Sound,X&Y,2005-04-18,how long before i get in before it starts befo...,0.010,0.974,0.016,0.1655,0.060037
34,Coldplay,Green Eyes,A Rush of Blood to the Head,2002-08-26,honey you are a rock upon which i stand and i ...,0.034,0.930,0.036,0.0772,0.148337
5,Coldplay,A Sky Full of Stars,Ghost Stories,2014-05-02,'cause you're a sky 'cause you're a sky full o...,0.088,0.822,0.090,0.0485,0.177037
23,Coldplay,Ink,Ghost Stories,2014-05-16,got a tattoo that said together thru life carv...,0.144,0.711,0.145,0.0258,0.199737
61,Coldplay,Oceans,Ghost Stories,2014-05-16,wait for your call love the call never came re...,0.168,0.690,0.141,0.4291,0.203563
88,Coldplay,Colour Spectrum,A Head Full of Dreams,2015-12-04,ambience poem because each has been sent as a...,0.000,1.000,0.000,0.0000,0.225537


In [10]:
title = new_df['title'].to_list()
album = new_df['album'].to_list()

Generating Playlist of songs and albums based on sentiment similarilty!

In [11]:
for i in range(20):
  print('Title: ' + title[i] + ', Album: ' + album[i])

Title: The Scientist, Album: A Rush of Blood to the Head
Title: WOTW / POTP, Album: Everyday Life
Title: Always in My Head, Album: Ghost Stories
Title: All I Can Think About Is You, Album: Kaleidoscope [EP]
Title: Speed of Sound, Album: X&Y
Title: Green Eyes, Album: A Rush of Blood to the Head
Title: A Sky Full of Stars, Album: Ghost Stories
Title: Ink, Album: Ghost Stories
Title: Oceans, Album: Ghost Stories
Title: Colour Spectrum, Album: A Head Full of Dreams
Title: Sunrise, Album: Everyday Life
Title: Warning Sign, Album: A Rush of Blood to the Head
Title: Til Kingdom Come, Album: X&Y
Title: Parachutes, Album: Parachutes
Title: Charlie Brown, Album: Mylo Xyloto
Title: Swallowed in the Sea, Album: X&Y
Title: Life in Technicolor ii, Album: Prospekt’s March [EP]
Title: Red Wedding, Album: Game of Thrones: The Musical
Title: Rastafarian Targaryen, Album: Game of Thrones: The Musical
Title: Church, Album: Everyday Life
